In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from catboost import CatBoostClassifier

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
#df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop(['client_id'], axis=1, inplace=True)
df.drop('slctn_nmbr', axis=1, inplace=True)
df.drop(['mgd_accum_period'], axis=1, inplace=True)
df.drop(['pmnts_sum_per_qrtr', 'pmnts_sum_per_year'], axis=1, inplace=True)
df.drop(['pmnts_sum'], axis=1, inplace=True)
df.drop(['lst_pmnt', 'pmnts_nmbr_per_year'], axis=1, inplace=True)
df.drop(['postal_code'], axis=1, inplace=True)
df.region = df.region.fillna('ЛИПЕЦКАЯ ОБЛ')
df.frst_pmnt_date = df.frst_pmnt_date.fillna('0')
df.lst_pmnt_date_per_qrtr = df.lst_pmnt_date_per_qrtr.fillna('0')
df.drop('npo_account_id', axis=1, inplace=True)

In [12]:
df1.shape

(633434, 31)

In [18]:
df1.head()

,slctn_nmbr,client_id,npo_account_id,npo_accnts_nmbr,pmnts_type,year,quarter,gender,age,clnt_cprtn_time_d,...,email,lk,assignee_npo,assignee_ops,postal_code,region,citizen,fact_addrss,appl_mrkr,evry_qrtr_pmnt
0,0,0x589BC912CB844D468212ADD42EAE4151,0xAA9E8DDF1310724995598EA2B42D7D87,2,1,2017,2017Q2,1,54,6595,...,0,1,-1,-1,398046.0,ЛИПЕЦКАЯ ОБЛ,1,1,0,1
1,2,0xCCF22420C90AE4459A25781A9A30F445,0x3B9B09857D152F468A42C5DEE6D723F7,1,3,2021,2021Q4,1,75,7078,...,-1,-1,-1,-1,0.0,МОСКВА Г,1,1,0,0
2,3,0x943D2C768A4FB38311E6605EEA2B9E4A,0xA7F12C768A4FB38311E835E4A7632E00,1,2,2015,2015Q3,1,35,1675,...,0,-1,-1,-1,666211.0,ИРКУТСКАЯ ОБЛ,-1,1,0,0
3,2,0x99E64FD29CC3B04189C5C9EBEAF6F9A4,0xBFE4BDFBE0E037478C29BA4F38121B15,1,1,2015,2015Q3,1,50,5180,...,-1,-1,-1,-1,162610.0,ВОЛОГОДСКАЯ ОБЛ,1,1,0,1
4,0,0x654EEEBA731C4A4881816631229C5B65,0x4B5C7804E403C842B4ECBF69BC3EDD7C,2,1,2011,2011Q3,-1,46,296,...,0,-1,-1,-1,427111.0,УДМУРТСКАЯ РЕСП,1,1,0,1


In [7]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.csv")
df1.drop(['client_id'], axis=1, inplace=True)
df1.drop('slctn_nmbr', axis=1, inplace=True)
df1.drop(['mgd_accum_period'], axis=1, inplace=True)
df1.drop(['pmnts_sum_per_qrtr', 'pmnts_sum_per_year'], axis=1, inplace=True)
df1.drop(['pmnts_sum'], axis=1, inplace=True)
df1.drop(['lst_pmnt', 'pmnts_nmbr_per_year'], axis=1, inplace=True)
df1.drop(['postal_code'], axis=1, inplace=True)
df1.region = df.region.fillna('ЛИПЕЦКАЯ ОБЛ')
df1.frst_pmnt_date = df.frst_pmnt_date.fillna('0')
df1.lst_pmnt_date_per_qrtr = df.lst_pmnt_date_per_qrtr.fillna('0')
df1.drop('npo_account_id', axis=1, inplace=True)
ob_cols = df1.select_dtypes('object').columns.to_list()

In [8]:
df.groupby('region')['churn'].mean()

region
-                     0.035714
2 МКР                 0.000000
77                    0.000000
АБЗЕЛИЛОВСКИЙ         0.000000
АБХАЗИЯ               0.000000
                        ...   
ЯМАЛО-НЕНЕЦКИЙ АО.    0.000000
ЯМАЛО-НЕНЦКИЙ АО      0.058824
ЯНАУЛЬСКИЙ            0.010234
ЯНАУЛЬСКИЙ Р-Н        0.000000
ЯРОСЛАВСКАЯ ОБЛ       0.006075
Name: churn, Length: 448, dtype: float64

In [9]:
df1.groupby('region')['quarter'].count()

region
-                        7
77                       2
АБЗЕЛИЛОВСКИЙ            2
АБХАЗИЯ                  9
АДЫГЕЯ РЕСП            225
                      ... 
ЯМАЛО-НЕНЕЦКИЙ АО.      21
ЯМАЛО-НЕНЦКИЙ АО         5
ЯНАУЛЬСКИЙ             144
ЯНАУЛЬСКИЙ Р-Н          22
ЯРОСЛАВСКАЯ ОБЛ       1867
Name: quarter, Length: 399, dtype: int64

In [10]:
for col in ob_cols:
    if df1[col].nunique() < 5:
        one_hot = pd.get_dummies(df1[col], prefix=col, drop_first=True)
        df1 = pd.concat((df1.drop(col, axis=1), one_hot), axis=1)
    else:
        mean_target = df.groupby(col)['churn'].mean()
        df1[col] = df1[col].map(mean_target)


In [ ]:
X

In [13]:
X = df1.copy()

In [35]:
model_xgb = xgb.XGBClassifier()

model_xgb.load_model("/content/drive/MyDrive/Colab Notebooks/xgboost_model.json")

In [14]:
from_file = CatBoostClassifier()

from_file.load_model("/content/drive/MyDrive/Colab Notebooks/cboost")

In [15]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.csv")

In [20]:
submission_id = df1.npo_account_id
submission_quarter = df1.quarter

In [21]:
n_predictions = from_file.predict(X)

In [36]:
n_predictions = model_xgb.predict(X)

In [37]:
n_predictions.shape

(633434,)

In [42]:
output = pd.DataFrame({'npo_account_id': submission_id,
                       'quarter' : submission_quarter,
                       'churn': n_predictions})

output.head()

,npo_account_id,quarter,churn
0,0xAA9E8DDF1310724995598EA2B42D7D87,2017Q2,0
1,0x3B9B09857D152F468A42C5DEE6D723F7,2021Q4,0
2,0xA7F12C768A4FB38311E835E4A7632E00,2015Q3,0
3,0xBFE4BDFBE0E037478C29BA4F38121B15,2015Q3,0
4,0x4B5C7804E403C842B4ECBF69BC3EDD7C,2011Q3,0


In [16]:
sample_submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_submission.csv')

In [40]:
sample_submission_df.drop(['churn'], axis=1, inplace=True)

In [43]:
sample_submission_df = sample_submission_df.merge(output, how='left', on=['npo_account_id', 'quarter'])

In [47]:
sample_submission_df.churn = 1

In [48]:
sample_submission_df

,npo_account_id,quarter,churn
0,0x0000132B2D126446B3E105530BA834B9,2009Q2,1
1,0x0000132B2D126446B3E105530BA834B9,2012Q1,1
2,0x0000132B2D126446B3E105530BA834B9,2017Q1,1
3,0x0000132B2D126446B3E105530BA834B9,2017Q2,1
4,0x0000132B2D126446B3E105530BA834B9,2020Q2,1
...,...,...,...
633429,0xFFFF57AF7B56DE4891B4CD4E78A5B892,2020Q4,1
633430,0xFFFF5DA61B60574194D4B977023B15B9,2009Q4,1
633431,0xFFFF5DA61B60574194D4B977023B15B9,2017Q4,1
633432,0xFFFFF59735AAF14E9DABA46C84CAD40B,2017Q3,1


In [50]:
sample_submission_df.to_csv('sample_submission.csv')

In [32]:
sample_submission_df.churn.unique()

array([0, 1])

In [ ]:
X = df.drop('churn', axis=1)
Y = df['churn']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0, test_size=0.2)

In [ ]:
cboost = CatBoostClassifier(depth=6, iterations=400, learning_rate=0.1, verbose=0)
cboost.fit(X_train,
           Y_train)
y_pred = cboost.predict(X_test)

print(classification_report(Y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9920    0.9985    0.9952    706848
           1     0.8367    0.4900    0.6181     11136

    accuracy                         0.9906    717984
   macro avg     0.9144    0.7443    0.8067    717984
weighted avg     0.9896    0.9906    0.9894    717984



In [ ]:
xgb_model = xgb.XGBClassifier(max_depth=9, n_estimators=200, learning_rate=0.1)
xgb_model.fit(X_train, Y_train)
y_pred = xgb_model.predict(X_test)

print(classification_report(Y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9927    0.9984    0.9955    706848
           1     0.8360    0.5314    0.6498     11136

    accuracy                         0.9911    717984
   macro avg     0.9143    0.7649    0.8226    717984
weighted avg     0.9902    0.9911    0.9901    717984



In [ ]:
xgb_model.save_model('xgboost_model.json')

In [ ]:
cboost.save_model('cboost')